# Pretrained Model, classification, Elastic Net

In [1]:
import os
import pandas as pd
import numpy as np
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Innolab/Github') 

Mounted at /content/drive


In [2]:
from src.preprocessing.preprocessing_matlab_files import preprocess_mat_files
from src.preprocessing.data_preparation import prepare_data

In [3]:
matlab_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/Matlab Files" # Enter the directory for the matlab files
excel_path = "/content/drive/MyDrive/Colab Notebooks/Innolab/DELCODE_dataset.xlsx" # Enter the directory for the corresponding excel sheet
write_dir = "/content/drive/MyDrive/Colab Notebooks/Innolab/write" # ...
export_file = False # rename to export file
preprocessing_type = 'aggregation' 

df = preprocess_mat_files(matlab_dir = matlab_dir, excel_path = excel_path, write_dir = write_dir,
                           export_file = export_file, preprocessing_type = preprocessing_type, statistic = 'mean')

loading files
Starting Preprocessing
Creating Final Dataset
Done!


In [4]:
df

,ConnID,Repseudonym,siteid,age,visdat,sex,prmdiag,edyears,MEM_score,Apoe,IDs,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
0,1,0a8d02f2b,11,66,17.08.2016,0,2,17,0.054016,0.0,1.0,0.059918,-0.038175,0.013602,-0.007439,0.072988,0.035254,0.005983,0.021569,0.538193,0.359326,0.409272,0.099061,0.155599,0.219386,0.161535,0.621305,0.430063,0.333243,0.092059,0.213101,0.191816,0.532068,0.174604,0.106723,0.326073,0.221852,0.322626,0.020247,0.159319,0.150326,0.175592,0.081184,0.105608,0.355734,0.317396,0.349668
1,2,0a71a953d,17,72,30.03.2015,0,1,20,-0.468749,1.0,2.0,0.10496,-0.027038,0.020629,-0.002622,0.019785,-0.023587,0.012025,0.052178,0.507799,0.359254,0.361147,0.049819,0.130366,-0.137414,0.04217,0.671531,0.473377,0.367554,0.140397,-0.002806,0.095852,0.459918,0.188397,0.030068,0.023321,0.034736,0.384345,0.135988,0.102555,0.07141,0.322519,-0.043426,0.090363,0.192952,0.105353,0.240881
2,3,0a61339db,11,72,13.05.2015,1,1,16,-0.093521,0.0,3.0,0.07133,0.005331,0.012895,0.026959,0.023896,0.026039,0.033056,0.021831,0.199761,0.074975,0.071457,-0.01712,0.052231,0.00726,0.049056,0.303302,0.190495,0.17687,0.03085,0.020871,-0.008662,0.266599,0.142112,0.027551,0.117675,-0.00927,0.203192,-0.014111,0.051064,-0.016667,0.145192,0.057935,0.06542,0.152252,0.080831,0.19634
3,4,0b28aed58,17,76,18.01.2016,0,1,20,0.466027,0.0,4.0,0.188084,0.052211,-0.111568,-0.09,0.029541,0.036005,0.035995,0.083385,0.482514,0.024467,-0.107833,0.03448,0.060489,-0.116487,0.093149,0.313839,0.182987,0.212667,-0.025838,-0.009318,-0.052479,0.246719,0.081127,-0.046371,0.104513,-0.019481,0.277193,-0.021194,0.038132,-0.001597,0.12639,-0.001889,0.064197,0.17999,0.118052,0.227498
4,5,0c1c5ae77,8,64,12.03.2015,1,1,13,1.452099,0.0,5.0,0.132788,0.04202,0.09332,0.060415,0.066898,0.022056,0.051668,0.080887,0.205802,-0.009584,0.094284,0.054277,0.037151,0.15134,0.174541,0.238345,0.136062,0.16104,-0.017533,0.086181,0.09898,0.252812,0.137191,-0.008658,0.242225,0.125819,0.191427,-0.021946,0.161507,0.138011,0.1411,0.009321,0.092701,0.30901,0.235893,0.37739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
904,906,f816d269a,16,84,2017-11-21,2,3,18,-2.388095,0.0,906.0,0.136389,-0.023649,-0.002348,-0.07102,0.079082,0.033848,0.001036,0.069726,0.545994,0.194554,0.221084,0.132955,-0.041355,0.089987,0.066649,0.27206,0.221954,0.15329,-0.021906,0.093227,0.013773,0.247335,0.072557,-0.043635,0.122286,-0.007813,0.180578,-0.003443,0.078697,0.061495,0.229756,0.035669,0.053695,0.155652,0.11343,0.156167
905,907,f98419539,5,71,2014-09-11,2,3,13,-1.860675,1.0,907.0,0.178495,-0.096916,0.058761,-0.039244,0.122438,0.031171,0.042034,0.026155,0.290446,-0.056688,-0.056182,-0.144704,0.02513,-0.05151,-0.009278,0.225529,0.038229,0.20374,-0.01182,-0.006614,0.016697,0.253735,0.013468,-0.005859,0.067822,-0.058574,0.410099,-0.045411,0.088022,0.08374,0.220006,0.010572,0.065844,0.178566,0.066199,0.169795
906,908,fade343c8,8,78,2015-11-23,2,3,8,-1.868184,1.0,908.0,0.097265,-0.026466,-0.028496,-0.019094,0.018134,0.067899,0.024621,0.035075,0.178263,0.056536,0.053847,-0.008416,0.00087,-0.012427,0.012002,0.161124,0.091493,0.08265,0.008871,-0.021607,-0.041121,0.147687,0.024451,-0.011598,0.081368,0.016668,0.137266,0.002639,-0.02314,-0.028287,0.112552,-0.016535,-0.012336,0.127273,0.094503,0.102097
907,909,fbc8a48a0,14,61,2017-03-20,2,3,9,-2.432351,0.0,909.0,0.131525,-0.043982,0.079655,-0.051607,0.117778,0.101683,0.019386,0.078948,0.163418,-0.094566,0.056415,-0.041263,-0.068523,0.020028,-0.034854,0.430648,0.096152,0.308294,0.093714,-0.058784,-0.10513,0.167568,0.071621,-0.072117,0.017321,-0.143462,0.324986,0.095204,0.018929,-0.069679,0.194042,0.028932,0.10987,0.101379,0.069668,0.249134


In [5]:
classification = True
columns_drop = ["ConnID", "Repseudonym", "siteid", "visdat", "MEM_score", "Apoe", "IDs"]
target = "prmdiag"
y_0 = [0]
y_1 = [2, 3]
train_size = 0.8
seed = 123
split = True

In [6]:
# preparation of data
ytrain, Xtrain, ytest, Xtest = prepare_data(data = df, classification = classification,
                         columns_drop = columns_drop, target = target, y_0 = y_0, y_1 = y_1,
                         train_size = train_size, seed = seed, split = split)


pd.concat([ytest, Xtest], axis = 1) # test data

,y,age,sex,edyears,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,1_1,1_2,1_3,1_4,1_5,1_6,1_7,2_2,2_3,2_4,2_5,2_6,2_7,3_3,3_4,3_5,3_6,3_7,4_4,4_5,4_6,4_7,5_5,5_6,5_7,6_6,6_7,7_7
30,0.0,1.774171,-1.149848,-0.727818,0.434838,0.382307,0.303081,-0.097007,0.213827,1.043929,-2.062528,-0.917540,2.188176,0.942426,0.937939,0.617081,2.254855,-0.353408,0.693790,-0.544271,-0.256409,-0.839045,1.100152,0.081316,1.211944,-1.101850,-0.428872,0.596500,-0.140458,0.019555,-0.787740,0.283360,-0.483366,0.340769,1.871021,-0.268800,0.378994,0.241618,-1.326802,-0.826122
792,1.0,1.106080,-1.149848,2.001953,3.034160,-1.977782,1.698651,-2.596911,3.352848,2.994703,-1.822604,-0.378240,-0.460228,0.090870,0.562435,-1.369870,0.123149,-1.375625,-1.570714,0.966685,-0.347261,1.018120,3.174129,-2.349659,-1.358024,-0.583129,-2.111727,0.047910,-2.149766,-1.179574,0.793469,2.205271,-1.774523,-0.314803,1.380451,-3.034238,-2.042218,1.017431,1.212451,-0.092794
497,0.0,0.103945,0.469414,-0.386597,-1.050092,-0.758109,-1.383459,-0.403851,-1.090751,-0.078474,0.015509,-0.448443,2.083029,1.890783,0.675133,-0.908430,-1.064161,-1.113624,-0.209327,0.825771,0.238800,-1.000596,-0.390360,-0.883236,0.667346,1.509386,-0.004780,-0.448309,0.724293,0.640444,0.649550,0.710833,0.518457,0.199268,-0.716098,0.304737,0.178686,0.452410,-0.218096,-0.668842
594,1.0,1.440126,-1.149848,1.660732,0.350918,-0.899691,-0.921190,-0.832911,0.063839,-0.220817,-0.008640,0.220047,1.155308,0.919712,1.303066,1.320260,0.984257,1.427121,1.076986,-0.079730,0.441934,0.814687,0.799680,1.977812,1.460173,-0.185731,0.910081,0.797751,0.835447,2.219176,0.616446,1.511951,1.481339,1.157358,-0.064459,0.910829,0.508736,0.035485,1.003808,0.571721
712,0.0,0.772035,0.469414,-0.727818,-0.035243,0.454559,1.603328,1.148474,0.658342,0.379007,0.613534,0.789149,0.709565,-1.225142,-0.934996,-0.715944,0.839062,1.103586,1.496045,0.588564,1.072911,2.056021,0.711869,1.399302,0.395532,2.189284,1.691299,0.443938,0.798261,-0.790348,1.268149,-0.009855,0.924348,0.213277,-0.085519,0.612997,0.275452,-0.031656,1.103737,1.907031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,0.0,-0.898191,-1.149848,0.978289,0.419945,1.961219,0.827395,1.079114,0.683160,0.651626,-0.053118,0.474594,3.511502,2.450484,1.030613,1.168287,1.325694,-0.071624,1.766849,1.204363,1.363293,0.591428,0.092971,0.800295,1.780789,-0.287906,0.798875,-0.464912,0.507183,1.936979,0.437365,-0.307840,0.178323,0.649008,1.188442,-0.438207,-0.917173,-0.396943,0.721518,0.354325
332,1.0,0.605013,0.469414,-0.386597,0.573963,1.048515,0.584199,-0.393066,-0.689162,1.784377,-0.193461,-0.332308,0.557920,0.457824,-0.601892,-0.004032,0.684116,-0.354701,0.832071,-0.915950,-0.222422,-0.370188,0.434684,0.761316,1.217400,-0.372494,0.180248,-0.744354,-0.028886,0.423772,-1.052707,0.010457,-0.294123,-0.137458,1.020236,-0.336492,-0.233258,-1.609627,-1.069307,-1.661246
228,1.0,0.103945,0.469414,2.001953,-1.557824,0.715071,1.009846,-0.727829,0.524721,0.368626,-2.411631,-0.744229,-1.926542,-0.044548,-0.935482,0.372556,-1.422836,-1.013739,-1.211035,0.801229,-0.602876,1.360380,2.380142,-0.300459,0.584091,-1.908031,-0.896090,0.010392,-0.853922,0.905927,-1.204349,3.052047,-0.912856,0.570937,-1.058691,-0.092646,-0.234765,-1.662786,-0.931215,-2.204884
397,1.0,2.108216,-1.149848,-0.727818,-0.587617,-0.932513,-0.714712,-1.475907,-0.236595,-0.607174,-1.148728,-1.458467,-0.137338,-0.222383,0.961423,-0.536979,-1.112989,0.229220,-0.893908,-0.472945,-0.069024,-0.723601,-0.229725,-0.784141,-0.706633,0.147587,-1.725894,-0.611062,-1.494512,-0.689900,-1.060124,-0.426682,-1.100174,-0.711539,-1.525440,-0.254562,-0.248919,-0.169679,0.058756,-0.365943


# Train Model
Evaluate AUC and accuracy on test data

In [7]:
from src.models.pipeline_elastic_net import model_elastic_net
from sklearn.metrics import accuracy_score, roc_auc_score

In [8]:
model = model_elastic_net(X_train = Xtrain, y_train = ytrain, 
                          classification = True, 
                          n_alphas_logreg = 10, 
                          l1_ratios_logreg= np.linspace(0,  1, 11).tolist(), 
                          cv_logreg = 5,
                          max_iter = 500) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


In [9]:
predictions = model.predict(Xtest)
accuracy_score(ytest, predictions)

0.6914893617021277

In [10]:
pred_prob = model.predict_proba(Xtest)
auc = roc_auc_score(ytest, pred_prob[:, 1])
auc

0.706959706959707

In [11]:
model.l1_ratio_

array([0.1])

In [12]:
model.C_

array([0.04641589])

# Save Model

In [13]:
import pickle

In [14]:
filename ='pretrained_elnet_class_agg_mean.sav'
pickle.dump(model, open(filename, 'wb'))

# Visualisation Feature Importance and Coeffs (evtl Ridge)

In [ ]:
# load model
model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/Innolab/pretrained_elnet_class_agg_mean.sav", 'rb'))